In [ ]:
# Notebook parameters. Values here are for development only and
# will be overridden when running via snakemake and papermill.

config_file = "../../../config/afun.yaml"
alert_id = "SA-AFUN-01"

In [ ]:
from bokeh.io import output_notebook
from IPython.display import Markdown
from selection_atlas.setup import AtlasSetup
from selection_atlas.page_utils import AtlasPageUtils

# Initialise the atlas setup.
setup = AtlasSetup(config_file)
page_utils = AtlasPageUtils(setup=setup)

# Load the alert.
alert = page_utils.load_alert(alert_id)
region = alert["region"]
region_contig, region_span = region.split(":")
region_start, region_stop = region_span.replace(",", "").split("-")

# N.B., do not add the "remove-output" tag to this cell!!! If you do,
# the bokeh javascript libraries will not get loaded in the generated
# HTML page. The call to output_notebook() injects javascript in the
# cell output which triggers the bokeh javascript libraries to be loaded
# in the page.
output_notebook(hide_banner=True)

In [ ]:
Markdown(f"# Alert {alert_id} (*{alert['label']}*)")

In [ ]:
content = f"This alert reports selection signals on Chromosome {region_contig} within the region {region_span} bp."
Markdown(content)

## Selection signals

{term}`Selection signal`s overlapping this {term}`genome region` are shown in the figure below.

In [ ]:
df_signals = page_utils.load_signals(
    contig=region_contig,
    start=region_start,
    stop=region_stop,
)

gene_labels = dict()
for item in alert["ir_candidate_genes"]:
    g = item["identifier"]
    gene_labels[g] = " "

if len(df_signals) > 0:
    page_utils.plot_signals(
        df=df_signals,
        contig=region_contig,
        x_min=df_signals["span2_pstart"].min() - 50_000,
        x_max=df_signals["span2_pstop"].max() + 50_000,
        gene_labels=gene_labels,
        genes_height=90,
    )
else:
    display(Markdown("No signals found."))

## Cohorts affected

Overlapping {term}`selection signal`s are found in the following {term}`cohort`s. 

In [ ]:
cohorts_affected = df_signals["cohort_id"]
gdf_cohorts_affected = (
    page_utils.gdf_cohorts.set_index("cohort_id").loc[cohorts_affected].reset_index()
)
page_utils.plot_cohorts_map(
    gdf_cohorts=gdf_cohorts_affected,
    zoom=3,
    url_prefix="../",
)

In [ ]:
content = ""
for _, row in df_signals.sort_values("cohort_label").iterrows():
    cohort_id = row["cohort_id"]
    content += f"* {{doc}}`../cohort/{cohort_id}`\n"
Markdown(content)

In [ ]:
content = ""
if "ir_candidate_genes" in alert:
    content += """
## Insecticide resistance genes

The following {term}`gene`s are found within this {term}`genome region` and may be driving 
{term}`recent positive selection` based on evidence for an association with 
{term}`insecticide resistance`. Please note that other genes are also within the affected 
genome region and may be driving selection.
"""
    for gene in alert["ir_candidate_genes"]:
        gene_id = gene["identifier"]
        url = f"https://vectorbase.org/vectorbase/app/record/gene/{gene_id}"
        content += f"### <a href='{url}' target='_blank'>{gene_id}</a>"
        gene_names = gene["names"]
        if gene_names:
            content += " (" + ", ".join([f"*{n}*" for n in gene_names]) + ")"
        content += "\n\n"
        notes = gene["notes"]
        content += f"{notes} See also:\n\n"
        for reference in gene["references"]:
            citation = reference["citation"]
            url = reference["url"]
            title = reference["title"]
            content += f"* <a href='{url}' target='_blank'>{citation}</a> {title}\n"
        content += "\n"

Markdown(content)

In [ ]:
content = "\n"  # ensure some content, otherwise postprocessing breaks
if "see_also" in alert:
    content += """
## See also

"""
    for item in alert["see_also"]:
        content += f"* {item}\n"

Markdown(content)